In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense,LSTM, SpatialDropout1D,Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, Input 
from keras.layers.normalization import BatchNormalization

from sklearn.ensemble import ExtraTreesClassifier

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim import utils

from nltk.tokenize import word_tokenize


%matplotlib inline

Using TensorFlow backend.
C:\Users\saroj\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
def extract(coinname):
    df = pd.read_csv('{}.csv'.format(coinname))
    df = df.replace(np.inf, np.nan)
    df = df.dropna(axis=0)
    
    return df

In [3]:
BTC = extract('BTCUSD')

In [4]:
BTC['Dec'] = BTC['Topic'] + BTC['Text']

In [5]:
ETH =extract('ETHUSD')

In [6]:
ETH['Dec'] = ETH['Topic'] + ETH['Text']

In [7]:
LTC =extract('LTCUSD')

In [8]:
LTC['Dec'] = LTC['Topic'] + LTC['Text']

In [9]:
XRP =extract('XRPUSD')

In [10]:
XRP['Dec'] = XRP['Topic'] + XRP['Text']

In [11]:
DASH =extract('DASHUSD')

In [12]:
DASH['Dec'] = DASH['Topic'] + DASH['Text']

In [13]:
DOGE =extract('DOGEUSD')

In [14]:
DOGE['Dec'] = DOGE['Topic'] + DOGE['Text']

In [15]:
STR =extract('STRUSD')

In [16]:
STR['Dec'] = STR['Topic'] + STR['Text']

In [17]:
XMR =extract('XMRUSD')

In [18]:
XMR['Dec'] = XMR['Topic'] + XMR['Text']

In [19]:
df = pd.DataFrame(columns=['Dec',  'Direction'])

b = BTC[(BTC['Direction'] == 'Long') | (BTC['Direction'] == 'Short')].reset_index(drop=True)[['Dec', 'Direction']]

e = ETH[(ETH['Direction'] == 'Long') | (ETH['Direction'] == 'Short')].reset_index(drop=True)[['Dec', 'Direction']]

l = LTC[(LTC['Direction'] == 'Long') | (LTC['Direction'] == 'Short')].reset_index(drop=True)[['Dec', 'Direction']]

d = DASH[(DASH['Direction'] == 'Long') | (DASH['Direction'] == 'Short')].reset_index(drop=True)[['Dec', 'Direction']]

x = XRP[(XRP['Direction'] == 'Long') | (XRP['Direction'] == 'Short')].reset_index(drop=True)[['Dec', 'Direction']]

do = DOGE[(ETH['Direction'] == 'Long') | (DOGE['Direction'] == 'Short')].reset_index(drop=True)[['Dec', 'Direction']]

s = STR[(ETH['Direction'] == 'Long') | (STR['Direction'] == 'Short')].reset_index(drop=True)[['Dec', 'Direction']]

xm = XMR[(XMR['Direction'] == 'Long') | (XMR['Direction'] == 'Short')].reset_index(drop=True)[['Dec', 'Direction']]




df = pd.concat([df, b, e, l, d, x, do, s, xm ])

C:\Users\saroj\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  del sys.path[0]
C:\Users\saroj\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app


In [20]:
df = df.reset_index(drop=True)

In [21]:
labeledRow = []

In [22]:
for idx, row in df['Dec'].iteritems():
    labeledRow.append(TaggedDocument(utils.to_unicode(row).split(), ['Text' + '_%s' % str(idx)]))

In [23]:
docModel = Doc2Vec(min_count=0, window=5, vector_size=150, sample=1e-4, negative=5, workers=4, epochs=5, seed=1)

In [24]:
docModel.build_vocab(labeledRow)

In [25]:
docModel.train(labeledRow, total_examples=docModel.corpus_count, epochs=docModel.iter)

C:\Users\saroj\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


In [26]:
X = np.zeros((df.shape[0], 150))

In [27]:
for i in range(df.shape[0]):
    X[i] = docModel.docvecs['Text_'+str(i)]

In [28]:
df['Direction'] = df['Direction'].replace('Long', 1)

In [29]:
df['Direction'] = df['Direction'].replace('Short', 0)

In [30]:
Y = df['Direction']

In [31]:
X.shape

(3485, 150)

In [32]:
Y.shape

(3485,)

In [33]:
Xtrain = np.asarray(X[:3000])

In [34]:
Ytrain = np.asarray(Y[:3000])

In [35]:
Xtest = np.asarray(X[3000:])

In [36]:
Ytest = np.asarray(Y[3000:])

In [37]:
#create model
visible = Input(shape=(1, 150))

hidden1 = LSTM(128, return_sequences=True)(visible)
hidden1 = LSTM(128, return_sequences=True)(hidden1)


hidden1  = Conv1D(filters=128, kernel_size=1, activation= 'relu')(hidden1)
hidden1 = MaxPooling1D(pool_size= 3, padding='same')(hidden1)

hidden1 = BatchNormalization()(hidden1)

hidden1 = Dropout(2.0)(hidden1)

hidden1  = Conv1D(filters=64, kernel_size=1, activation= 'relu')(hidden1)
hidden1 = MaxPooling1D(pool_size= 2, padding='same')(hidden1)

hidden1 = BatchNormalization()(hidden1)

hidden1 = Flatten()(hidden1)

hidden1 = Dropout(2.0)(hidden1)

hidden1 = Dense(64, activation='relu')(hidden1)

hidden1 = Dense(32, activation='relu')(hidden1)

hidden1 = Dense(16, activation='relu')(hidden1)

output = Dense(1, activation='sigmoid')(hidden1)

model = Model(inputs=visible, outputs=output)


In [38]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1, 150)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 1, 128)            142848    
_________________________________________________________________
lstm_2 (LSTM)                (None, 1, 128)            131584    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1, 128)            16512     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1, 128)            0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 1, 128)            512       
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 128)            0         
__________

In [39]:
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [40]:
#Fitting model
model.fit(Xtrain.reshape(-1, 1,150 ), Ytrain, epochs=50, batch_size=64, validation_data=(Xtest.reshape(-1, 1,150 ), Ytest))

Train on 3000 samples, validate on 485 samples
Epoch 1/50
3000/3000 [==============================] - 5s 2ms/step - loss: 0.3754 - acc: 0.8540 - val_loss: 0.5449 - val_acc: 0.7196
Epoch 2/50
3000/3000 [==============================] - 1s 208us/step - loss: 0.1701 - acc: 0.9463 - val_loss: 0.5645 - val_acc: 0.7546
Epoch 3/50
3000/3000 [==============================] - 1s 213us/step - loss: 0.1007 - acc: 0.9737 - val_loss: 0.5579 - val_acc: 0.7423
Epoch 4/50
3000/3000 [==============================] - 1s 196us/step - loss: 0.0964 - acc: 0.9707 - val_loss: 0.5072 - val_acc: 0.8144
Epoch 5/50
3000/3000 [==============================] - 1s 198us/step - loss: 0.0978 - acc: 0.9693 - val_loss: 0.4261 - val_acc: 0.7835
Epoch 6/50
3000/3000 [==============================] - 1s 250us/step - loss: 0.0832 - acc: 0.9763 - val_loss: 0.9899 - val_acc: 0.5711
Epoch 7/50
3000/3000 [==============================] - 1s 234us/step - loss: 0.0917 - acc: 0.9733 - val_loss: 7.4705 - val_acc: 0.3113
Epo

In [41]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)


In [42]:
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk
